# **라이브러리**

In [215]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup

In [240]:
train = pd.read_csv(r'C:\Users\JH\Desktop\Final\CBA\Data\데이터정리중\재무변수_한국은행_Altman.csv',index_col=0)
# train = train.drop(['감사의견코드','공시제목','년'], axis=1)
train

dataset = pd.read_csv(r'C:\Users\JH\Desktop\Final\CBA\Data\kind_bankrupt_Final.csv', index_col = 0)
print(train.shape, dataset.shape)

(27357, 21) (938, 6)


# **부도 추가 정의**

In [241]:
train.loc[(train['부도'] == 1) & (train['소속코드'] == 4)]  # 얘네들은 공시를 찾을 수 없거나 다른 사유에 의해서 부도

,회사명,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자기자본배당률,금융비용 대 부채비율,자기자본배율,유동부채비율,...,유동자산회전률,총자본투자효율,순운전자본비율,누적수익성비율,총자산영업이익률,총자산회전율,년,key,공시제목,부도
89,(주)강원랜드,35250.0,2000/12,4.0,2003/09/04,UQ,4.97,0.000000,461.26,47.46,...,0.97,55.64,36.96,0.111201,0.130214,0.265266,2001.0,352502001,0,1.0
90,(주)강원랜드,35250.0,2001/12,4.0,2003/09/04,UQ,4.97,0.003868,461.26,47.46,...,0.97,55.64,36.96,0.335832,0.435723,0.664572,2002.0,352502002,0,1.0
91,(주)강원랜드,35250.0,2002/12,4.0,2003/09/04,UQ,6.94,0.002562,638.00,28.26,...,1.13,43.26,22.25,0.498198,0.362327,0.578167,2003.0,352502003,0,1.0
109,(주)경윤하이드로에너지,19120.0,2000/12,4.0,2011/04/06,UQ,3.49,0.064542,500.94,54.92,...,2.14,27.08,38.11,0.477951,0.074341,1.596995,2001.0,191202001,0,1.0
110,(주)경윤하이드로에너지,19120.0,2001/12,4.0,2011/04/06,UQ,3.07,0.041816,548.78,51.69,...,2.15,28.52,42.51,0.499863,0.085689,1.452628,2002.0,191202002,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27296,후야인포넷(주),32050.0,2000/12,4.0,2005/04/26,UQ,0.00,0.032603,654.91,37.11,...,1.85,13.97,49.69,0.081179,0.022520,1.017036,2001.0,320502001,0,1.0
27297,후야인포넷(주),32050.0,2001/12,4.0,2005/04/26,UQ,0.00,0.021178,360.12,33.95,...,1.24,-38.25,40.44,-0.444249,-0.191165,1.288789,2002.0,320502002,0,1.0
27298,후야인포넷(주),32050.0,2002/12,4.0,2005/04/26,UQ,0.00,0.021223,114.67,190.59,...,1.38,-42.34,11.37,0.000000,-0.368169,0.968728,2003.0,320502003,0,1.0
27299,후야인포넷(주),32050.0,2003/12,4.0,2005/04/26,UQ,0.00,0.206503,10.75,190.14,...,0.35,-252.34,-2.14,-3.185847,-1.857084,0.996953,2004.0,320502004,0,1.0


In [242]:
droplist = ['5호','인프라','네트','C&C','솔믹스','E&M','Metal','필룩스']
idxbox = []
for i in droplist:
    idx = dataset.loc[dataset['거래소코드'] == i].index[0]
    idxbox.append(idx)

dataset.drop(index= idxbox, axis=0, inplace=True)


In [243]:
dataset['거래소코드'] = dataset['거래소코드'].astype(int)

In [244]:
subset = train.loc[(train['부도'] == 1) & (train['소속코드'] == 4)]
subset['거래소코드'].unique().astype(int)

subset2 = pd.DataFrame({'거래소코드' : subset['거래소코드'].unique().astype(int)})
subset2

,거래소코드
0,35250
1,19120
2,58900
3,51530
4,35780
...,...
591,198940
592,58480
593,26180
594,38960


In [245]:
dataset['거래소코드'].isin(subset2['거래소코드']).value_counts()

True     612
False    318
Name: 거래소코드, dtype: int64

# **시장조치 공시가 TS2000데이터에도 존재 여부**

In [246]:
dataset['TS'] = dataset['거래소코드'].isin(subset2['거래소코드'])

In [247]:
dataset.reset_index(inplace=True, drop=True)

In [248]:
textbox = []
for i in range(len(dataset)):
    html = dataset['text'][i]
    soup = BeautifulSoup(html, 'html.parser').get_text()
    
    if len(html[soup.find('5.'):soup.find('6.')]) > 1:
        textbox.append(html[soup.find('5.'):soup.find('6.')])
    else:
        textbox.append(html[soup.find('2.'): soup.find('3.')])


In [249]:
textbox2 = []
for i in range(len(textbox)):
    a = textbox[i].split('\n')
    try:
        textbox2.append(a[1])
    except:
        textbox2.append('')

In [250]:
dataset['폐지사유'] = textbox
dataset['폐지사유요약'] = textbox2
dataset

,회사명,거래소코드,공시발생일,공시제목,text,link,TS,폐지사유,폐지사유요약
0,주은리스,30970,2002-11-28 18:54,상장폐지,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/28/K00...,False,5.등록취소사유\n액면가액일정비율 미달로 연속하여 10일간 지속\n\n\n,액면가액일정비율 미달로 연속하여 10일간 지속
1,글로벌파워,64910,2002-11-26 18:53,상장폐지,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/26/K00...,False,5.등록취소사유\n증권투자회사법 제64조의 규정에 의한 해산사유의 발생\n\n\n,증권투자회사법 제64조의 규정에 의한 해산사유의 발생
2,주은리스,30970,2002-11-21 18:24,상장폐지승인,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/21/K00...,False,5.등록취소사유\n액면가액일정비율 미달로 연속하여 10일간 지속\n\n\n,액면가액일정비율 미달로 연속하여 10일간 지속
3,정산애강,22220,2002-11-13 16:59,상장폐지,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/13/K00...,False,5.등록취소사유\n최종부도 및 은행거래정지\n\n\n,최종부도 및 은행거래정지
4,소프트윈,49790,2002-11-06 18:09,상장폐지((주)소프트윈),\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/06/K00...,True,5.등록취소사유\n최종부도 및 은행거래정지\n\n\n,최종부도 및 은행거래정지
...,...,...,...,...,...,...,...,...,...
925,유안타제2호스팩,219960,2018-02-08 17:51,상장폐지,\n\n\n\n\n\n\r\n\t.PGBRK\t\t\t{page-break-afte...,https://kind.krx.co.kr/external/2018/02/08/001...,False,5.상장폐지사유\n상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사...,상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사유 미해소
926,셀트리온,68270,2018-02-07 15:48,상장폐지(유가증권시장 상장),\n\n\n\n\n\n\r\n\t.PGBRK\t\t\t{page-break-afte...,https://kind.krx.co.kr/external/2018/02/07/000...,True,5.상장폐지사유\n유가증권시장 상장\n\n\n,유가증권시장 상장
927,키움스팩3호,218710,2018-02-01 18:08,상장폐지,\n\n\n\n\n\n\r\n\t.PGBRK\t\t\t{page-break-afte...,https://kind.krx.co.kr/external/2018/02/01/000...,False,5.상장폐지사유\n상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사...,상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사유 미해소
928,골든브릿지제3호스팩,219580,2018-01-25 18:10,상장폐지,\n\n\n\n\n\n\r\n\t.PGBRK\t\t\t{page-break-afte...,https://kind.krx.co.kr/external/2018/01/25/000...,False,5.상장폐지사유\n상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사...,상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사유 미해소


In [251]:
# dataset['key'] = dataset['거래소코드'].astype(str) + (dataset.공시발생일.str[0:4].astype(int) + 1).astype(str)
dataset['key'] = dataset['거래소코드'].astype(str) + (dataset.공시발생일.str[0:4].astype(int)).astype(str)
dataset.head()

,회사명,거래소코드,공시발생일,공시제목,text,link,TS,폐지사유,폐지사유요약,key
0,주은리스,30970,2002-11-28 18:54,상장폐지,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/28/K00...,False,5.등록취소사유\n액면가액일정비율 미달로 연속하여 10일간 지속\n\n\n,액면가액일정비율 미달로 연속하여 10일간 지속,309702002
1,글로벌파워,64910,2002-11-26 18:53,상장폐지,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/26/K00...,False,5.등록취소사유\n증권투자회사법 제64조의 규정에 의한 해산사유의 발생\n\n\n,증권투자회사법 제64조의 규정에 의한 해산사유의 발생,649102002
2,주은리스,30970,2002-11-21 18:24,상장폐지승인,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/21/K00...,False,5.등록취소사유\n액면가액일정비율 미달로 연속하여 10일간 지속\n\n\n,액면가액일정비율 미달로 연속하여 10일간 지속,309702002
3,정산애강,22220,2002-11-13 16:59,상장폐지,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/13/K00...,False,5.등록취소사유\n최종부도 및 은행거래정지\n\n\n,최종부도 및 은행거래정지,222202002
4,소프트윈,49790,2002-11-06 18:09,상장폐지((주)소프트윈),\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/06/K00...,True,5.등록취소사유\n최종부도 및 은행거래정지\n\n\n,최종부도 및 은행거래정지,497902002


In [252]:
train['key'] = train['key'].astype(str)

In [253]:
train = train.merge(dataset[['공시발생일','공시제목','text','link','TS','폐지사유','폐지사유요약','key']], on='key', how= 'outer')
train.shape

(27802, 28)

In [254]:
train.head(1)

,회사명,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자기자본배당률,금융비용 대 부채비율,자기자본배율,유동부채비율,...,key,공시제목_x,부도,공시발생일,공시제목_y,text,link,TS,폐지사유,폐지사유요약
0,(주)CMG제약,58820.0,2000/12,5.0,0,UQ,0.0,0.019074,179.37,131.99,...,588202001,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# **정리중**

In [255]:
train[['공시발생일','공시제목_y','text','link','폐지사유','폐지사유요약','TS']] = train[['공시발생일','공시제목_y','text','link','폐지사유','폐지사유요약','TS']].fillna(0)

In [256]:
train = train.dropna()
train.shape

(27322, 28)

In [257]:
train.to_csv(r'C:\Users\JH\Desktop\Final\CBA\Data\최종데이터셋\Finaldataset.csv', index = False)

# **부도 사유 상세 정리**

In [258]:
dataset = pd.read_csv(r'C:\Users\JH\Desktop\Final\CBA\Data\최종데이터셋\Finaldataset.csv')

In [259]:
dataset = dataset.rename(columns={'공시제목_x':'정기공시제목','공시제목_y':'수시공시제목'})
dataset

,회사명,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자기자본배당률,금융비용 대 부채비율,자기자본배율,유동부채비율,...,key,정기공시제목,부도,공시발생일,수시공시제목,text,link,TS,폐지사유,폐지사유요약
0,(주)CMG제약,58820.0,2000/12,5.0,0,UQ,0.00,0.019074,179.37,131.99,...,588202001,0,1.0,0,0,0,0,0,0,0
1,(주)CMG제약,58820.0,2001/12,5.0,0,UQ,0.00,0.023517,179.37,131.99,...,588202002,0,1.0,0,0,0,0,0,0,0
2,(주)CMG제약,58820.0,2002/12,5.0,0,UQ,0.00,0.043081,133.37,132.37,...,588202003,0,1.0,0,0,0,0,0,0,0
3,(주)CMG제약,58820.0,2003/12,5.0,0,UQ,0.00,0.058531,131.30,71.46,...,588202004,0,1.0,0,0,0,0,0,0,0
4,(주)CMG제약,58820.0,2004/12,5.0,0,UQ,0.00,0.047346,52.25,101.56,...,588202005,0,1.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27317,흥구석유(주),24060.0,2016/12,5.0,0,UQ,2.10,0.012651,4611.84,5.19,...,240602017,0,1.0,0,0,0,0,0,0,0
27318,흥구석유(주),24060.0,2017/12,5.0,0,UQ,2.08,0.012502,4650.29,12.83,...,240602018,0,1.0,0,0,0,0,0,0,0
27319,흥구석유(주),24060.0,2018/12,5.0,0,UQ,2.48,0.018056,4696.15,5.40,...,240602019,0,1.0,0,0,0,0,0,0,0
27320,흥구석유(주),24060.0,2019/12,5.0,0,UQ,2.83,0.000300,5133.42,6.05,...,240602020,0,1.0,0,0,0,0,0,0,0


In [262]:
subset = dataset.loc[dataset['수시공시제목'] != '0',['회사명','거래소코드','회계년도','소속코드','감사의견코드','정기공시제목','수시공시제목','폐지사유요약','공시발생일','부도']]

In [263]:
subset.loc[subset['부도'] == 1]

,회사명,거래소코드,회계년도,소속코드,감사의견코드,정기공시제목,수시공시제목,폐지사유요약,공시발생일,부도
91,(주)강원랜드,35250.0,2002/12,4.0,UQ,0,상장폐지,증권거래소 상장,2003-09-02 16:14,1.0
119,(주)경윤하이드로에너지,19120.0,2010/12,4.0,UQ,0,상장폐지,기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지 기준에 해당한다고 결정,2011-03-24 18:01,1.0
174,(주)골드카운티,58900.0,2009/12,4.0,UQ,0,상장폐지,"기업의 계속성, 경영의 투명성 또는 기타 코스닥시장의 건전성 등을 종합적으로 고려하...",2010-09-29 17:59,1.0
354,(주)굿이엠지,51530.0,2008/12,4.0,UQ,0,상장폐지,주된 영업의 정지에 의한 상장폐지기준 해당,2009-10-21 18:09,1.0
358,(주)그로웰텔레콤,35780.0,2003/12,4.0,UQ,0,상장폐지,액면가액일정비율미만(취소사유발생일 : 2004년 12월 24일),2004-12-27 17:05,1.0
...,...,...,...,...,...,...,...,...,...,...
26827,한솔신텍(주),99660.0,2017/12,4.0,UQ,0,상장폐지,발행한 어음 또는 수표가 주거래은행에 의하여 최종부도로 결정되거나 거래은행에 의한 ...,2018-06-26 18:45,1.0
27150,현대정보기술(주),26180.0,2018/12,4.0,UQ,0,상장폐지(피흡수합병),피흡수합병,2019-07-12 16:00,1.0
27175,현주컴퓨터(주),38960.0,2004/06,4.0,UQ,0,상장폐지,최종부도(상장폐지사유 발생일 : 2005년 04월 25일),2005-04-26 17:57,1.0
27264,후야인포넷(주),32050.0,2004/12,4.0,UQ,0,상장폐지,자본전액잠식(상장폐지사유 발생일 : 2005년 04월 11일),2005-04-12 16:46,1.0


In [264]:
subset.to_csv(r'C:\Users\JH\Desktop\Final\CBA\Data\데이터정리중\시장조치부도판단ver2.csv', index=False)